<a href="https://colab.research.google.com/github/DeepikaS04/chefineer/blob/main/GPT2%20trained%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install transformers[torch]

In [9]:
import pandas as pd
import numpy as np
import re

In [4]:
def cleaning(s):
    s = str(s)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    s = s.replace("co","")
    s = s.replace("https","")
    s = s.replace("[\w*"," ")
    return s

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Food.csv")

In [ ]:
df['Protein'].fillna(value = 'Low Protein')

In [ ]:
df.drop(['Unnamed: 0'], axis = 1)

In [16]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [17]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator


def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset
  )

  trainer.train()
  trainer.save_model()

In [18]:
train_file_path = "/content/drive/MyDrive/Colab Notebooks/Food.csv"
model_name = 'gpt2'
output_dir = '/content/results'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 5.0
save_steps = 500

In [19]:
# !pip install accelerate

In [20]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.377300
1000,2.017800
1500,1.909400
2000,1.834900
2500,1.784700
3000,1.746600
3500,1.711200
4000,1.653500
4500,1.663700
5000,1.628200


In [23]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [24]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(cuisine, course, diet, time, protein, ingredients, max_length):
    model_path = "/content/results"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{cuisine, course, diet, time, protein, ingredients}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))
    print(tokenizer.decode(final_outputs, skip_special_tokens=True))

In [27]:
cuisine = input()
course = input()
diet = input()
time = input()
protein = input()
ingredients = input()
max_len = 500
generate_text(cuisine, course, diet, time, protein, ingredients, max_len)

Italian
Snack
Non Vegetarian
50
High Protein
Pasta cheese
('Italian', 'Snack', 'Non Vegetarian', '50', 'High Protein', 'Pasta cheese')),4,Greek Yogurt,4 Dry Red Chillies,10,25,4,Greek,Appetizer,Vegetarian,,"To begin making the Greek Yogurt and Vegetable Smoothie Recipe, get all the ingredients prepped and ready.Blend the yogurt with 1/2 cup of water, salt in a blender to smooth and fluffy mixture. Keep it aside.In a sauce pan, add in the cooked spinach, olive oil, tomato puree, chilli flakes, thyme and milk. Blend the mixture to make a smooth puree. Add the cream and milk to this mix and stir until incorporated.You can add a little warm water if the mixture you need for a more creamy mixture. Once blended, you can transfer the spinach puree to a serving bowl.Heat a saucepan, over medium heat, add in the cheese and top it with the cooked spinach puree.Fold the spinach puree in half, check for salt and adjust. Garnish with basil leaves and serve the Greek Yogurt Smoothie with a dollop of

<br>
<h1> Now Using Bard API to get the results.
<br>

In [ ]:
# !pip install bardapi

In [ ]:
from bardapi import Bard
bard = Bard(token_from_browser = True)
res = bard.get_answer("get me a continental dish I can make in 30 mins with ingredients and recipe")['content']
print(res)

In [ ]:
from bardapi import Bard
import os
import requests
os.environ['_BARD_API_KEY'] = 'Ywiq1nvAr1BrAQXITbCtJOV6hxLg8ijrqFdmJ6rQs3rUbsduCEGoVzRPrFMmMy0A-jCFbQ.'
# token='xxxxxxx'

session = requests.Session()
session.headers = {
            "Host": "bard.google.com",
            "X-Same-Domain": "1",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
            "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8",
            "Origin": "https://bard.google.com",
            "Referer": "https://bard.google.com/",
        }
session.cookies.set("__Secure-1PSID", os.getenv("_BARD_API_KEY"))
# session.cookies.set("__Secure-1PSID", token)

bard = Bard(session=session, timeout=30)
print(bard.get_answer("get me a continental dish I can make in 30 mins with ingredients and recipe")['content'])

# Continued conversation without set new session
# print(bard.get_answer("What is my last prompt??")['content'])

Sure, here is a continental dish that you can make in 30 minutes:

**Chicken Piccata**

Ingredients:

* 1 pound boneless, skinless chicken breasts, cut into 1-inch pieces
* 1/4 cup all-purpose flour
* 1/4 teaspoon salt
* 1/4 teaspoon black pepper
* 1 tablespoon olive oil
* 1/4 cup dry white wine
* 1/4 cup chicken broth
* 2 tablespoons capers, drained
* 1 tablespoon lemon juice

Instructions:

1. Pound the chicken breasts to 1/4-inch thickness.
2. In a shallow dish, combine the flour, salt, and pepper. Dredge the chicken in the flour mixture.
3. Heat the olive oil in a large skillet over medium heat. Add the chicken and cook for 2-3 minutes per side, or until golden brown and cooked through.
4. Remove the chicken from the skillet and set aside.
5. Add the wine, broth, capers, and lemon juice to the skillet. Bring to a boil, then reduce heat to low and simmer for 5 minutes, or until the sauce has thickened.
6. Return the chicken to the skillet and cook for 1-2 minutes, or until heated th